def normalize_layers(self):
        for i,w in enumerate(self.weights):
            self.weights[i] = (w - w.mean(axis=0)) / w.std(axis=0)
        for i,b in enumerate(self.biases):
            self.biases[i] = (b - b.mean(axis=0)) / b.std(axis=0)

In [ ]:
from evoframe.reward_builders import RewardBuilderGame
from evoframe.population_update_builders import PopulationUpdateBuilderStatic
from evoframe.selector_function import SelectorFunctionFactory
from evoframe import PopulationManager
from evoframe.models import FeedForwardNetwork
from evoframe.models import ActivationFunctions
from evoframe.games import Game
from evoframe import get_agent_wrapper_func
import evoframe.func_with_context as fwc

import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from evoframe.experiment_results import plot_rewards, show_best_fnn_weights, get_best_model_of_epoch

%matplotlib inline

In [ ]:
import collections

def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

In [ ]:
import imageio

def to_gif(experiment_name, num_epochs, duration=0.5):
    for i in range(1, num_epochs+1):
        print(i)
        f = show_best_fnn_weights(experiment_name, i)
        b = f.to_image(format="png")
        with open("images/image{}.png".format(i), "wb") as fi:
            fi.write(b)
    images = []
    filenames = ["images/image{}.png".format(i) for i in range(1, num_epochs+1)]
    for i,filename in enumerate(filenames):
        print(i)
        images.append(imageio.imread(filename))
    imageio.mimsave('nn.gif', images, duration=duration)

# Guess the point

In [ ]:
from evoframe.games import GuessPoint

In [ ]:
# Define global context
context = recursively_default_dict()

# Game
game_creation_func = lambda context: GuessPoint(np.array([0.2,0.8,0.5]), np.array([0.4, 0.5, 10]))
game_creation_func = fwc.func_with_context(game_creation_func, context=context)

# Model
layer_sizes = [3, 5, 3]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_relu(), ActivationFunctions.get_id())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Game-Model interface
predict_func = lambda model, inputs: model.predict(inputs)
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_func) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_context(context) \
.get()

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 1, 0.3, 0.3) \
.add_operator("es_2_crossover", 0.01, 0.8) \
.add_operator("es_1_copy", 0.01) \
.add_operator("es_n_rewards_gradient", 0.01) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 500
pickle_models_after_gens = 1
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, pickle_models_after_gens, context)

# Run population manager
num_epochs = 10
experiment_name = "guesspoint"
pm.run(num_epochs, experiment_name)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
to_gif(experiment_name, num_epochs)

![...](nn.gif "segment")

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

# Guess the function

In [ ]:
from evoframe.games import GuessFunction

In [ ]:
# Define global context
context = recursively_default_dict()

# Game
game_func = lambda i: np.array([2*i[0]-3*i[1]+4, i[1]-8*i[2]-5])
input_dim = 3
input_domains = [(-1,1),(-1,1),(3,7)]
sample_every = [0.1, 0.1, 0.3]
game_creation_function = lambda context: GuessFunction(game_func, input_dim, input_domains, sample_every)
game_creation_function = fwc.func_with_context(game_creation_function, context=context)

# Model
layer_sizes = [3, 5, 2]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_arctan(), ActivationFunctions.get_id())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Game-Model interface
predict_func = lambda model, inputs: model.predict(inputs)
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_function) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_context(context) \
.get()

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 0.2, 0.1, 0.1) \
.add_operator("es_1_mutation", 0.2, 0.1, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.3, 0.3) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.1) \
.add_operator("es_2_crossover", 0.1, 0.8) \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient", 0.1) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 50
pickle_models_after_gens = 1
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, pickle_models_after_gens, context)

# Run population manager
num_epochs = 50
experiment_name = "guessfunction"
pm.run(num_epochs, experiment_name)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
to_gif(experiment_name, num_epochs)

In [ ]:
from evoframe.experiment_results import get_best_model_of_epoch

In [ ]:
test_array = np.array([0.5,0.5,5])
get_best_model_of_epoch(experiment_name, num_epochs).predict(test_array), game_func(test_array)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

# Play Tris, 9 inputs

In [ ]:
from evoframe.reward_builders.reward_builder_game import TournamentMode

In [ ]:
from evoframe.games import Tris

In [ ]:
# Define global context
context = recursively_default_dict()

# Game
game_creation_function = lambda context: Tris()
game_creation_function = fwc.func_with_context(game_creation_function, context=context)

# Model
layer_sizes = [9, 5, 5, 9]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_sigmoid(), ActivationFunctions.get_sigmoid())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Game-Model interface
def predict_func(model, inputs):
    # In Tris, 'inputs' is a 3x3 np.array, thus we must flatten it to a 9x1 np.array
    # output is a 9x1 vector, where the cell with highest value corresponding
    # to a valid move will be the chosen move
    return model.predict(inputs.flatten())[0]
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
# Since Tris is a two-players game, we can compute the reward with a tournament against the current generation
keep_only = 20
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_function) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_competitive_tournament(TournamentMode.VS_BEST_OF_EACH_GEN) \
.with_keep_only(keep_only) \
.with_context(context) \
.get()

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 0.2, 0.1, 0.1) \
.add_operator("es_1_mutation", 0.2, 0.1, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.3, 0.3) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.1) \
.add_operator("es_2_crossover", 0.1, 0.8) \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient", 0.1) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 50
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, keep_only, context)

# Run population manager
num_epochs = 400
experiment_name = "tris"
pm.run(num_epochs, experiment_name)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
to_gif(experiment_name, num_epochs, duration=0.04)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

In [ ]:
class AgentHuman:
    def __init__(self):
        pass
    
    def predict(self, board):
        move = int(input("Select move: "))
        prediction = [1 if i == move else 0 for i in range(9)]
        return prediction

In [ ]:
game = Tris()
game.play(agent_wrapper_func(get_best_model_of_epoch(experiment_name, num_epochs)), AgentHuman(), interactive=True)

# Play Tris, 18 inputs

In [ ]:
# Define global context
context = recursively_default_dict()

# Game
game_creation_function = lambda context: Tris()
game_creation_function = fwc.func_with_context(game_creation_function, context=context)

# Model
layer_sizes = [18, 9, 9]
get_model_func = lambda context: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_sigmoid(), ActivationFunctions.get_sigmoid())
get_model_func = fwc.func_with_context(get_model_func, context=context)

# Game-Model interface
def predict_func(model, inputs):
    # In Tris, 'inputs' is a 3x3 np.array
    # output is a 9x1 vector, where the cell with highest value corresponding
    # to a valid move will be the chosen move
    board = inputs
    inputs = []
    for row in board:
        for cell in row:
            if cell == Tris.PLAYER_1:
                inputs += [1, 0]
            elif cell == Tris.PLAYER_2:
                inputs += [0, 1]
            else:
                inputs += [0, 0]
    return model.predict(np.array(inputs))[0]
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
# Since Tris is a two-players game, we can compute the reward with a tournament against the current generation
keep_only = 30
reward_function = RewardBuilderGame() \
.with_game_creation_function(game_creation_function) \
.with_agent_wrapper_func(agent_wrapper_func) \
.with_competitive_tournament(TournamentMode.VS_BEST_OF_EACH_GEN) \
.with_keep_only(keep_only) \
.with_context(context) \
.get()

# Update population function
get_new_pop_f = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 0.2, 0.1, 0.1) \
.add_operator("es_1_mutation", 0.2, 0.1, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.3, 0.3) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.1) \
.add_operator("es_2_crossover", 0.1, 0.8) \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient", 0.1) \
.add_selector_f(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.with_context(context) \
.get()

# Population manager
pop_size = 30
pm = PopulationManager(pop_size, get_model_func, reward_function, get_new_pop_f, keep_only, context)

# Run population manager
num_epochs = 300
experiment_name = "tris_18"
pm.run(num_epochs, experiment_name)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

In [ ]:
to_gif(experiment_name, num_epochs, duration=0.04)

In [ ]:
class AgentHuman:
    def __init__(self):
        pass
    
    def predict(self, board):
        move = int(input("Select move: "))
        prediction = [1 if i == move else 0 for i in range(9)]
        return prediction

In [ ]:
game = Tris()
game.play(agent_wrapper_func(get_best_model_of_epoch(experiment_name, num_epochs)), AgentHuman(), interactive=True)